In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from imblearn.over_sampling import ADASYN
from skfeature.function.similarity_based import fisher_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn import metrics

#import data from csv
ad = pd.read_csv('D:\Spring2017\Artificial Intelligence\RTEP3OesoAvant2_DiseaseFree.csv')
ad.head()


#get all the features
X_data = ad.iloc[:, 0:29]
X = pd.DataFrame(X_data)

#Print the colums name
print("Features before feature selection: {}".format(X.columns.values))

#Get classes
y_data = ad['Label']
y = pd.DataFrame(y_data)
y=y.values.ravel()



#Save the resmapling data into npy
X_resampled = np.load('free_x1.npy')
y_resampled = np.load('free_y1.npy')

X_resampled = pd.DataFrame(X_resampled)
X_resampled.columns = X.columns.values
X_resampled['Sex=M'] = X_resampled['Sex=M'].astype('bool')
X_resampled['OMS'] = X_resampled['OMS'].astype('bool')
X_resampled['Stade=ND'] = X_resampled['Stade=ND'].astype('bool')
X_resampled['Stade={2,2a,2b}'] = X_resampled['Stade={2,2a,2b}'].astype('bool')
X_resampled['Stade=3'] = X_resampled['Stade=3'].astype('bool')
X_resampled['1/3 sup'] = X_resampled['1/3 sup'].astype('bool')
X_resampled['1/3 moyen'] = X_resampled['1/3 moyen'].astype('bool')
X_resampled['1/3 inf'] = X_resampled['1/3 inf'].astype('bool')
X_resampled['Dysphagie=0'] = X_resampled['Dysphagie=0'].astype('bool')
X_resampled['Dysphagie=1'] = X_resampled['Dysphagie=1'].astype('bool')
X_resampled['Dysphagie=2'] = X_resampled['Dysphagie=2'].astype('bool')
X_resampled['Dysphagie=3'] = X_resampled['Dysphagie=3'].astype('bool')

cv=StratifiedKFold(n_splits=8)

for train, test in cv.split(X_resampled, y_resampled):
    score = fisher_score.fisher_score(X_resampled.iloc[train], y_resampled[train])

print(score)
idx = fisher_score.feature_ranking(score)

X1 = X_resampled.iloc[:, [idx[0], idx[1], idx[2], idx[3], idx[4], idx[5], idx[6], idx[7], idx[8], idx[9], idx[10], idx[11]]]

#X1 = X_resampled.iloc[:, [idx[0], idx[1], idx[2], idx[3], idx[4],idx[5]]]

#print(X_resampled.columns.values)

X_resampled = X1

clf = DecisionTreeClassifier(random_state=0)
cv=StratifiedKFold(n_splits=8)

scores = cross_val_score(clf, X_resampled, y_resampled, scoring="roc_auc", cv=cv)
acc = cross_val_score(clf, X_resampled, y_resampled, scoring="accuracy", cv=cv)
recall = cross_val_score(clf, X_resampled, y_resampled, scoring="recall", cv=cv)
prec = cross_val_score(clf, X_resampled, y_resampled, scoring="precision", cv=cv)

#acc = cross_val_score(classifier, X_resampled, y_resampled, scoring="accuracy", cv=cv)
#print(scores)
print("Accuracy {}".format(acc.mean()))
print("AUC {}".format(scores.mean()))
print("Recall {}".format(recall.mean()))
print("Precision {}".format(prec.mean()))

for train, test in cv.split(X_resampled, y_resampled):
    probas_ = clf.fit(X_resampled.iloc[train], y_resampled[train]).predict_proba(X_resampled.iloc[test])

export_graphviz(clf, out_file="freeTree_Fisher.dot", class_names = [ "non-Disease_Free", "Disease_Free"] ,
                feature_names=X.columns.values, impurity=False, filled=True
                )

with open("freeTree_Fisher.dot") as f:
    dot_graph = f.read()

graphviz.Source(dot_graph)
